q-query vector: What I am looking for
k-key vector: What I can offer
v-value vector: what I actually offer 

In [13]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
     

In [2]:
seq_len=4 #length of sentence
batch_size=1
input_dim=512
d_model=512
x=torch.randn((batch_size,seq_len,input_dim))

Note that x is not the value of the input but the value that we get after input embedding. In this case its taken rand

In [3]:
x.size()

torch.Size([1, 4, 512])

In [5]:
#done to create q k and v vectors concatinated
qkv_layer=nn.Linear(input_dim,3*d_model)
qkv_layer

Linear(in_features=512, out_features=1536, bias=True)

In [6]:
qkv=qkv_layer(x)
qkv

tensor([[[-0.1451,  1.3501,  1.1285,  ...,  0.5574, -0.4863, -0.6798],
         [ 0.2442, -1.0424, -0.0583,  ..., -0.4577,  0.4065,  0.1546],
         [-0.9975,  0.1986, -0.1687,  ..., -0.9849, -0.3517, -0.6339],
         [ 0.9389, -0.5585,  1.2884,  ..., -0.3924,  0.2324,  0.0486]]],
       grad_fn=<ViewBackward0>)

In [7]:
qkv.shape

torch.Size([1, 4, 1536])

In [9]:
num_heads=8
head_dim=d_model //num_heads
qkv=qkv.reshape(batch_size,seq_len,num_heads,3*head_dim)

In [10]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [11]:
qkv=qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [12]:
q,k,v=qkv.chunk(3,dim=-1)
q.shape , k.shape , v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

# self attention


In [15]:
d_k=q.size()[-1]
scaled=torch.matmul(q,k.transpose(-2,-1))/ math.sqrt(d_k)
scaled.shape , q.shape, k.transpose(-2,-1).shape

(torch.Size([1, 8, 4, 4]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 64, 4]))

In [16]:
mask=torch.full(scaled.size(),float('-inf'))
mask=torch.triu(mask,diagonal=1)
mask

tensor([[[[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf],
          [0., 0., -inf, -inf],
          [0., 0., 0., -inf],
          [0., 0., 0.,

In [23]:
(scaled+mask)[0][0]

tensor([[ 0.1988,    -inf,    -inf,    -inf],
        [ 0.0669,  0.0051,    -inf,    -inf],
        [-0.2175,  0.0596, -0.2197,    -inf],
        [ 0.0235, -0.3644,  0.0372, -0.1524]], grad_fn=<SelectBackward0>)

In [24]:
scaled=scaled+mask

In [25]:
attention=F.softmax(scaled,dim=-1)

In [26]:
attention

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000],
          [0.5154, 0.4846, 0.0000, 0.0000],
          [0.3015, 0.3977, 0.3008, 0.0000],
          [0.2832, 0.1921, 0.2871, 0.2375]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.4864, 0.5136, 0.0000, 0.0000],
          [0.2335, 0.4124, 0.3542, 0.0000],
          [0.3559, 0.2032, 0.3044, 0.1364]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.5932, 0.4068, 0.0000, 0.0000],
          [0.5446, 0.1379, 0.3175, 0.0000],
          [0.2160, 0.2580, 0.3220, 0.2040]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.4644, 0.5356, 0.0000, 0.0000],
          [0.2361, 0.4223, 0.3416, 0.0000],
          [0.3288, 0.2533, 0.1679, 0.2501]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.4354, 0.5646, 0.0000, 0.0000],
          [0.2372, 0.4132, 0.3496, 0.0000],
          [0.2230, 0.3422, 0.1056, 0.3293]],

         [[1.0000, 0.0000, 0.0000, 0.0000],
          [0.6937, 0.3063, 0.0000, 0.0000],
          [0.5090, 0.3

In [27]:
values=torch.matmul(attention,v)
values.shape

torch.Size([1, 8, 4, 64])

# Function

In [28]:
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [29]:

values, attention = scaled_dot_product(q, k, v, mask=mask)

In [30]:

attention.shape
     

torch.Size([1, 8, 4, 4])

In [31]:

attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5154, 0.4846, 0.0000, 0.0000],
        [0.3015, 0.3977, 0.3008, 0.0000],
        [0.2832, 0.1921, 0.2871, 0.2375]], grad_fn=<SelectBackward0>)

In [32]:

values.size()

torch.Size([1, 8, 4, 64])

In [33]:

values = values.reshape(batch_size, seq_len, num_heads * head_dim)
values.size()

torch.Size([1, 4, 512])

In [34]:

linear_layer = nn.Linear(d_model, d_model)

In [35]:

out = linear_layer(values)

In [36]:
out

tensor([[[ 0.1513,  0.1235,  0.2849,  ..., -0.2099,  0.5354, -0.0492],
         [ 0.0939, -0.3143,  0.1780,  ...,  0.0615,  0.1062,  0.3315],
         [-0.4269, -0.1941,  0.2890,  ...,  0.4284, -0.0590, -0.0972],
         [ 0.2177,  0.3734,  0.1209,  ..., -0.2225, -0.0388,  0.0582]]],
       grad_fn=<ViewBackward0>)